In [18]:
import pandas as pd
import yfinance as yf
import warnings

from support import fama_french

from sklearn.linear_model import LinearRegression, Lasso

In [19]:
aapl = yf.download("AAPL", interval="1mo", progress=False)
aapl.index = pd.to_datetime(aapl.index)
aapl = aapl.dropna()
ff = fama_french(from_web=True)

def set_first(df, inplace=False):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        df.loc[:,"Date"] = [f"{y}-{m}" for y, m in zip(df.index.year.to_list(), df.index.month.to_list())]
        df = df.reset_index(drop=True)
        df.loc[:,"Date"] = pd.to_datetime(df.loc[:,"Date"])
        df = df.set_index("Date", drop=True)
        if not inplace:
            return df

ff = set_first(ff)
aapl = set_first(aapl)

aapl.loc[:,"Ret"] = aapl.loc[:,"Adj Close"].pct_change()

df = pd.merge(aapl.loc[:,"Ret"], ff, left_index=True, right_index=True)

df = df.dropna()

df.tail()

,Ret,Mkt_RF,SMB,HML,RMW,CMA,MOM,RF
Date,,,,,,,,
2022-04-01,-0.097131,-0.0946,-0.0040,0.0619,0.0363,0.0592,0.0489,0.0001
2022-05-01,-0.055883,-0.0034,-0.0006,0.0841,0.0144,0.0398,0.0248,0.0003
2022-06-01,-0.080080,-0.0843,0.0130,-0.0597,0.0185,-0.0470,0.0079,0.0006
2022-07-01,0.188634,0.0957,0.0187,-0.0410,0.0068,-0.0694,-0.0396,0.0008
2022-08-01,-0.032552,-0.0378,0.0151,0.0031,-0.0480,0.0131,0.0209,0.0019


In [3]:
mkt = yf.download("SPY", interval="1mo", progress=False)
mkt.index = pd.to_datetime(mkt.index)
mkt = mkt.dropna()
set_first(mkt)
mkt["Ret"] = mkt["Adj Close"].pct_change()
mkt = pd.merge(mkt.loc[:,"Ret"], ff, left_index=True, right_index=True)
mkt = mkt[["Ret", "RF"]]
mkt = pd.merge(aapl["Ret"], mkt, left_index=True, right_index=True)
mkt = mkt.dropna()
mkt = mkt.sub(mkt["RF"], axis=0).drop("RF", axis=1)
mkt.columns = ["AAPL", "MKT"]

mkt = mkt.last("5Y")

/var/folders/wh/63dtvrx97_31k2y9clt5xgwc0000gn/T/ipykernel_32309/3655766512.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mkt["Ret"] = mkt["Adj Close"].pct_change()


In [20]:
mkt = yf.download("SPY", interval="1mo", progress=False)
mkt.index = pd.to_datetime(mkt.index)
mkt = mkt.dropna()
set_first(mkt)
mkt["Ret"] = mkt["Adj Close"].pct_change()
mkt = pd.merge(mkt.loc[:,"Ret"], ff, left_index=True, right_index=True)
mkt = mkt[["Ret", "RF"]]
mkt = pd.merge(aapl["Ret"], mkt, left_index=True, right_index=True)
mkt = mkt.dropna()
mkt = mkt.sub(mkt["RF"], axis=0).drop("RF", axis=1)
mkt.columns = ["AAPL", "MKT"]

mkt = mkt.last("5Y")

/var/folders/wh/63dtvrx97_31k2y9clt5xgwc0000gn/T/ipykernel_32309/3655766512.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mkt["Ret"] = mkt["Adj Close"].pct_change()


In [21]:
df["Ret"] -= df["RF"]
data = df.last("5Y")
X, y = data.drop(["Ret", "RF"], axis=1), data[["Ret"]]

In [29]:
yahoo_lr = LinearRegression(fit_intercept=False).fit(mkt["MKT"].to_numpy().reshape(-1, 1), mkt["AAPL"].to_numpy().reshape(-1, 1))
fama_lr = LinearRegression(fit_intercept=False).fit(X, y)
yahoo_beta = yahoo_lr.coef_.flatten()[0]
fama_beta = fama_lr.coef_.flatten()[0]
print("Data as of October 13")
print("Actual Yahoo Beta 5Y Monthly: 1.25")
print(f"Calculated Yahoo Beta 5Y Monthly: {yahoo_beta:.2f}")
print(f"Calculated FF Beta 5Y Monthly: {fama_beta:.2f}")

Data as of October 13
Actual Yahoo Beta 5Y Monthly: 1.25
Calculated Yahoo Beta 5Y Monthly: 1.25
Calculated FF Beta 5Y Monthly: 1.35


In [31]:
print("CAPM\n")

print("With Intercept:")
x = pd.DataFrame(X.iloc[:,0])
int_cpt = True
linreg = LinearRegression(fit_intercept=int_cpt).fit(x, y)
betas = linreg.coef_.flatten().tolist()
betanames = X.columns.tolist()
for name, beta in zip(betanames, betas):
    print(f"{name} Beta: {beta:.2f}")
if int_cpt:
    print(f"Alpha: {linreg.intercept_[0]:.2f}")

print("\nWithout Intercept:")
x = pd.DataFrame(X.iloc[:,0])
int_cpt = False
linreg = LinearRegression(fit_intercept=int_cpt).fit(x, y)
betas = linreg.coef_.flatten().tolist()
betanames = X.columns.tolist()
for name, beta in zip(betanames, betas):
    print(f"{name} Beta: {beta:.2f}")
if int_cpt:
    print(f"Alpha: {linreg.intercept_[0]:.2f}")

CAPM

With Intercept:
Mkt_RF Beta: 1.18
Alpha: 0.02

Without Intercept:
Mkt_RF Beta: 1.24


In [15]:
print("3 Factor Model\n")

print("With Intercept:")
x = pd.DataFrame(X.iloc[:,:3])
int_cpt = True
linreg = LinearRegression(fit_intercept=int_cpt).fit(x, y)
betas = linreg.coef_.flatten().tolist()
betanames = X.columns.tolist()
for name, beta in zip(betanames, betas):
    print(f"{name} Beta: {beta:.2f}")
if int_cpt:
    print(f"Alpha: {linreg.intercept_[0]:.2f}")

print("\nWithout Intercept:")
x = pd.DataFrame(X.iloc[:,:3])
int_cpt = False
linreg = LinearRegression(fit_intercept=int_cpt).fit(x, y)
betas = linreg.coef_.flatten().tolist()
betanames = X.columns.tolist()
for name, beta in zip(betanames, betas):
    print(f"{name} Beta: {beta:.2f}")
if int_cpt:
    print(f"Alpha: {linreg.intercept_[0]:.2f}")

3 Factor Model

With Intercept:
Mkt_RF Beta: 1.26
SMB Beta: -0.38
HML Beta: -0.42
Alpha: 0.02

Without Intercept:
Mkt_RF Beta: 1.31
SMB Beta: -0.42
HML Beta: -0.43


In [16]:
print("5 Factor\n")

print("With Intercept:")
x = pd.DataFrame(X.iloc[:,:5])
int_cpt = True
linreg = LinearRegression(fit_intercept=int_cpt).fit(x, y)
betas = linreg.coef_.flatten().tolist()
betanames = X.columns.tolist()
for name, beta in zip(betanames, betas):
    print(f"{name} Beta: {beta:.2f}")
if int_cpt:
    print(f"Alpha: {linreg.intercept_[0]:.2f}")

print("\nWithout Intercept:")
x = pd.DataFrame(X.iloc[:,:5])
int_cpt = False
linreg = LinearRegression(fit_intercept=int_cpt).fit(x, y)
betas = linreg.coef_.flatten().tolist()
betanames = X.columns.tolist()
for name, beta in zip(betanames, betas):
    print(f"{name} Beta: {beta:.2f}")
if int_cpt:
    print(f"Alpha: {linreg.intercept_[0]:.2f}")

5 Factor

With Intercept:
Mkt_RF Beta: 1.24
SMB Beta: 0.02
HML Beta: -0.80
RMW Beta: 0.77
CMA Beta: 0.61
Alpha: 0.01

Without Intercept:
Mkt_RF Beta: 1.28
SMB Beta: 0.06
HML Beta: -0.88
RMW Beta: 0.87
CMA Beta: 0.74


In [17]:
print("5 Factor with Momentum\n")

print("With Intercept:")
x = X
int_cpt = True
linreg = LinearRegression(fit_intercept=int_cpt).fit(x, y)
betas = linreg.coef_.flatten().tolist()
betanames = X.columns.tolist()
for name, beta in zip(betanames, betas):
    print(f"{name} Beta: {beta:.2f}")
if int_cpt:
    print(f"Alpha: {linreg.intercept_[0]:.2f}")

print("\nWithout Intercept:")
x = X
int_cpt = False
linreg = LinearRegression(fit_intercept=int_cpt).fit(x, y)
betas = linreg.coef_.flatten().tolist()
betanames = X.columns.tolist()
for name, beta in zip(betanames, betas):
    print(f"{name} Beta: {beta:.2f}")
if int_cpt:
    print(f"Alpha: {linreg.intercept_[0]:.2f}")

5 Factor with Momentum

With Intercept:
Mkt_RF Beta: 1.31
SMB Beta: 0.15
HML Beta: -0.69
RMW Beta: 0.87
CMA Beta: 0.53
MOM Beta: 0.31
Alpha: 0.01

Without Intercept:
Mkt_RF Beta: 1.35
SMB Beta: 0.20
HML Beta: -0.75
RMW Beta: 0.97
CMA Beta: 0.64
MOM Beta: 0.34
